# LGBIO2060 - Exercise session 3
Kalman filtering or how to implement the prior and likelihood dynamically? 


## Import and helper functions
**Please execute the cell below to initialize the notebook environment**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
def plot_my_system(state_evolution):
  """
  Do not edit this function...

  the aim of this function is to represent the time-evolution of a dynamical linear system
  Author : Antoine de Comite 
  """
  fig, ax = plt.subplots(figsize=(8,6))
  xlim = None
  ylim = None
  ax.scatter(state_evolution[0,:],state_evolution[1,:],c='m',s=100,alpha=0.7)
  ax.plot(state_evolution[0,:],state_evolution[1,:],LineWidth=2,c='k')
  ax.set_xlabel("x_1")
  ax.set_ylabel("x_2")
  ax.set(xlim=xlim)
  ax.set(ylim=ylim)
  return ax


def plot_my_system_with_obs(state_evolution,obs_evolution):
  """
  Do not edit this function...

  the aim of this function is to represent the time-evolution of a dynamical linear system
  Author : Antoine de Comite 
  """
  fig, ax = plt.subplots(figsize=(8,6))
  xlim = None
  ylim = None
  ax.scatter(state_evolution[0,:],state_evolution[1,:],c='m',s=100,alpha=0.7)
  ax.plot(state_evolution[0,:],state_evolution[1,:],LineWidth=2,c='k',label='Latent state')
  ax.plot(obs_evolution[0,:],obs_evolution[1,:],LineWidth=2,c='g',label='Observation')
  ax.set_xlabel("x_1")
  ax.set_ylabel("x_2")
  ax.set(xlim=xlim)
  ax.set(ylim=ylim)
  ax.legend()
  return ax
  
import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

## Tutorial objectives 

At the end of this tutorial, you should be able to explain the concepts of dynamical system and continuous dynamic application of Bayes' theorem. 

# Section 1 - Dynamical systems

In this tutorial, we will be applying Bayes' theorem to dynamical systems. A dynamical system is a system for which the time-evolution of any point can be described by a time-dependent function. Many systems that we encounter in our daily life are dynamical systems: eyes movements, reaching movements, walking, population dynamics,... 


A dynamical system can be observed through some noise and uncertainty similarly to fixed variable (see the first two tutorials). Therefore, in order to obtain the best estimate of the latent state of the system, we can apply Bayes theorem at every time step. This is what we will do in this tutorial.


A dynamical system has the following continuous and discrete forms, respectively : 

\begin{eqnarray}
& & \\
\dot{x}\left(t\right) & = & A x\left(t\right) + \xi\left(t\right)\\
& & \\
x\left[t+1\right] & = & A x\left[t\right] + \xi\left[t\right] \\
& & \\
\end{eqnarray}

where $A$ is called the **state-transition matrix** matrix and $\xi\left(t\right)$ is Gaussian white noise generated from $\mathcal{N}\left(0,\Omega_{\text{motor}}\right)$. $\Omega_{\text{motor}}$ is the covariance matrix of the motor noise.

The latent state $x$ can have more than one entry, for example we may want to estimate the x- and y-positions of a fly. In this case, the latent state will become a latent state vector $x = \left[x_1,x_2 \right]^T$ and the dynamical can be written : 


\begin{eqnarray}
&&\\
\begin{bmatrix}
  x_1[t+1] \\
  x_2[t+1]
  \end{bmatrix} &=& \begin{bmatrix}A_{11} & A_{12}\\ A_{21} & A_{22} \end{bmatrix}\begin{bmatrix}x_1[t]\\ x_2[t]\end{bmatrix} + \begin{bmatrix}\xi_1[t] \\ \xi_2[t] \end{bmatrix}
  &&\\
\end{eqnarray}

**Exercise 1 - Implement a function that defines the system mentionned above**

Hints : 
* You can assume that the covariance matrix of the motor noise is diagonal and that all the entries of the diagonal are equal to $\sigma_{\text{motor}}$. **Hint**: Use the function numpy.random.multivariate_normal
* You will be performing matrix multiplications, consider this while implementing your function
* You can use the function *plot_my_system* to represent the time-evolution of your linear dynamical system





In [ ]:
def my_system(nsteps,x0,A,dt,sigma_motor):
  """
  my_system is a function that model the time-evolution of a linear dynamical system
  with state transition matrix A.
  Inputs : nsteps is the number of time steps to model
           x0 is the initial state (where we are starting from)
           A is the state-transition matrix
           dt is the time step used in the discretization
           sigma_motor is the std of the motor noise
  Outputs : state_evolution is a numpy array that contains the time-evolution of 
            the state vector
  """
  
  ######################
  ### your code here ###
  ######################

  
  return state_evolution

# Run the lines below to test your code

nsteps = 50
dt = 0.1
x0 = np.array([0,0]).T
A = np.array([[1., 1.], [-(2*np.pi/20.)**2., .9]])
sigma_motor = 0.05

######################
### your code here ###
######################

**What is the effect of the noise covariance on the behaviour of the system?**


Play with the widget below to explore this effect. 

In [ ]:
# @title
# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    sigma_motor=widgets.FloatSlider(0.05, min=0, max=1, step=0.05, layout=my_layout)

)

def sigma_motor(sigma_motor = 0.05):
    nsteps = 50
    dt = 0.1
    x0 = np.array([1,1]).T
    A = np.array([[1., 1.], [-(2*np.pi/20.)**2., .9]])
    state_evolution = my_system(nsteps,x0,A,dt,sigma_motor)
    plot_my_system(state_evolution)


---
However, as we saw in the previous tutorials, our sensory inputs are not perfect and it is impossible to know the real latent state; we can only get a rough estimate of it. Mathematically, we can define the observation of the latent state of our dynamical system as follows: 


$\begin{eqnarray}
& & \\
y[t]& = & H x[t] + \eta[t]\\
 & & \\
\begin{bmatrix}y_1[t]\\ y_2[t]\end{bmatrix} & = &\begin{bmatrix}1 & 0 \\ 0 & 1\end{bmatrix} \begin{bmatrix}x_1[t] \\ x_2[t]\end{bmatrix} + \begin{bmatrix}\eta_1[t]\\ \eta_2[t]\end{bmatrix}
& &\\
\end{eqnarray}$

where $H$ is the observation matrix and $\eta[t]$ is a vector of Gaussian white noise generated from $\mathcal{N}\left(0,\Omega_{\text{sensory}}\right)$. 

**Exercise 2 - Implement the observation of the latent state**

* you can use the function *plot_my_system_with_obs* for the plot

In [ ]:
def my_system_with_obs(nsteps,x0,A,H,dt,sigma_motor,sigma_sensory):
  """
  my_system_with_obs is a function that model the time-evolution of the latent state 
  and its observation
  """
  ######################
  ### your code here ###
  ######################

  return state_evolution,obs_evolution

# Run the lines below to test your code 
nsteps = 50
dt = 0.1
x0 = np.array([0,0]).T
A = np.array([[1.,1.],[-(2*np.pi/20.)**2.,0.9]])
H = np.eye(2)
sigma_motor = 0.05
sigma_sensory = 0.02
######################
### your code here ###
######################


**Compared to the first and second tutorials, what does the green line correspond to (in terms of Bayes' theorem?**

## Section 2 - Kalman filter or the optimal observation of continuous linear dynamical systems